# Treat cast date column

## Problem

a guy from a whatapp group:


“
Eu to tendo um problema em alterar o dado da tabela no databricks.... a tabela tem uma coluna chamada "DATE" e ela ta no formato string (com mês/dia/ano) eu quero mudar para formato data. Só que toda vez que faço isso a coluna inteira fica nula :/
“



## Solution 1

In [0]:
from pyspark.sql.functions import col, lit, when, to_date
from pyspark.sql import DataFrame, Column, SparkSession

spark: SparkSession = (
    SparkSession
    .builder
    .appName("Python Spark Date rule example")
    .getOrCreate()
)

source_df: DataFrame = spark.createDataFrame(
    [
        ("24/05/2025", "1"),
        ("31/01/2025", "2"),
        ("31/03/2025", "3"),
        ("28/02/2025", "4"),
        ("29/02/2025", "5"), # must return NULL (None) value
    ],
    ["date", "id"]
)

source_df.show(10,False)
source_df.printSchema()

date_format: str = "dd/MM/yyyy"
transformed_df: DataFrame = source_df.withColumn(
    "transformed_date",
    to_date(col("date"), date_format)
)

transformed_df.show(10,False)
transformed_df.printSchema()


+----------+---+
|date      |id |
+----------+---+
|24/05/2025|1  |
|31/01/2025|2  |
|31/03/2025|3  |
|28/02/2025|4  |
|29/02/2025|5  |
+----------+---+

root
 |-- date: string (nullable = true)
 |-- id: string (nullable = true)

+----------+---+----------------+
|date      |id |transformed_date|
+----------+---+----------------+
|24/05/2025|1  |2025-05-24      |
|31/01/2025|2  |2025-01-31      |
|31/03/2025|3  |2025-03-31      |
|28/02/2025|4  |2025-02-28      |
|29/02/2025|5  |null            |
+----------+---+----------------+

root
 |-- date: string (nullable = true)
 |-- id: string (nullable = true)
 |-- transformed_date: date (nullable = true)



## 'Solution 2'
The same solution as the previous one, but with:

`.config("spark.sql.legacy.timeParserPolicy", "CORRECTED")`

We can see that the last three dates in source_df are null.


In [0]:
from pyspark.sql.functions import col, lit, when, to_date
from pyspark.sql import DataFrame, Column, SparkSession

spark: SparkSession = (
    SparkSession
    .builder
    .appName("Python Spark Date rule example")
    .config("spark.sql.legacy.timeParserPolicy", "CORRECTED")
    .getOrCreate()
)

source_df: DataFrame = spark.createDataFrame(
    [
        ("24/05/2025", "1"),
        ("31/01/2025", "2"),
        ("31/03/2025", "3"),
        ("28/02/2025", "4"),
        ("29/02/2025", "5"), # must return NULL (None) value
        ("24/06/25", "6"), 
        ("24/5/25", "7"),
    ],
    ["date", "id"]
)

source_df.show(10,False)
source_df.printSchema()

date_format: str = "dd/MM/yyyy"
parsed_date_col: Column = to_date(col("date"),date_format)

transformed_df: DataFrame = source_df.withColumn(
    "transformed_date",
    to_date(col("date"), date_format)
)

transformed_df.show(10,False)
transformed_df.printSchema()


+----------+---+
|date      |id |
+----------+---+
|24/05/2025|1  |
|31/01/2025|2  |
|31/03/2025|3  |
|28/02/2025|4  |
|29/02/2025|5  |
|24/06/25  |6  |
|24/5/25   |7  |
+----------+---+

root
 |-- date: string (nullable = true)
 |-- id: string (nullable = true)

+----------+---+----------------+
|date      |id |transformed_date|
+----------+---+----------------+
|24/05/2025|1  |2025-05-24      |
|31/01/2025|2  |2025-01-31      |
|31/03/2025|3  |2025-03-31      |
|28/02/2025|4  |2025-02-28      |
|29/02/2025|5  |null            |
|24/06/25  |6  |null            |
|24/5/25   |7  |null            |
+----------+---+----------------+

root
 |-- date: string (nullable = true)
 |-- id: string (nullable = true)
 |-- transformed_date: date (nullable = true)



## Solution 3
Now we check the date formats using the configuration below
```
.config("spark.sql.legacy.timeParserPolicy", "CORRECTED")
```



In [0]:
from pyspark.sql.functions import col, lit, when, date_format, to_date, expr
from pyspark.sql import DataFrame, Column, SparkSession
from typing import List
from functools import reduce

spark: SparkSession = (
    SparkSession
    .builder
    .appName("Python Spark Date rule example")
    .config("spark.sql.legacy.timeParserPolicy", "CORRECTED")
    .getOrCreate()
)

mock_df: DataFrame = spark.createDataFrame(
    [
        ("24/05/2025", "1"),
        ("31/01/2025", "2"),
        ("31/03/2025", "3"),
        ("28/02/2025", "4"),
        ("29/02/2025", "5"), # must return NULL (None) value
        ("24/06/25", "6"), 
        ("24/5/25", "7"),
    ],
    ["date", "id"]
)
mock_df.show(10,False)
mock_df.printSchema()

date_formats: List[str] = [
    "dd/MM/yyyy",
    "dd/MM/yy",
    "d/M/yy"
    ]

date_column_name: str = "date"

sql_case_expr: str = reduce(
    lambda acc, fmt: acc + f"""
        WHEN date_format(to_date({date_column_name}, '{fmt}'), '{fmt}') = {date_column_name}
        THEN to_date({date_column_name}, '{fmt}')""",
    date_formats,
    "CASE"
)

sql_case_expr += f"""
    ELSE NULL
END"""

print(sql_case_expr)

transformed_df: DataFrame = mock_df.withColumn(
    "date_transformed", expr(sql_case_expr)
)

transformed_df.show(10,False)
transformed_df.printSchema()


+----------+---+
|date      |id |
+----------+---+
|24/05/2025|1  |
|31/01/2025|2  |
|31/03/2025|3  |
|28/02/2025|4  |
|29/02/2025|5  |
|24/06/25  |6  |
|24/5/25   |7  |
+----------+---+

root
 |-- date: string (nullable = true)
 |-- id: string (nullable = true)

CASE
        WHEN date_format(to_date(date, 'dd/MM/yyyy'), 'dd/MM/yyyy') = date
        THEN to_date(date, 'dd/MM/yyyy')
        WHEN date_format(to_date(date, 'dd/MM/yy'), 'dd/MM/yy') = date
        THEN to_date(date, 'dd/MM/yy')
        WHEN date_format(to_date(date, 'd/M/yy'), 'd/M/yy') = date
        THEN to_date(date, 'd/M/yy')
    ELSE NULL
END
+----------+---+----------------+
|date      |id |date_transformed|
+----------+---+----------------+
|24/05/2025|1  |2025-05-24      |
|31/01/2025|2  |2025-01-31      |
|31/03/2025|3  |2025-03-31      |
|28/02/2025|4  |2025-02-28      |
|29/02/2025|5  |null            |
|24/06/25  |6  |2025-06-24      |
|24/5/25   |7  |2025-05-24      |
+----------+---+----------------+

root
 |-

## Notes

### [Note - TimeParserPolicy](https://docs.databricks.com/aws/en/sql/language-manual/parameters/legacy_time_parser_policy)

### [Notes - Datetime Patterns for Formatting and Parsing](https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html)


There are several common scenarios for datetime usage in Spark:

CSV/JSON datasources use the pattern string for parsing and formatting datetime content.

Datetime functions related to convert StringType to/from DateType or TimestampType. For example, unix_timestamp, date_format, to_unix_timestamp, from_unixtime, to_date, to_timestamp, from_utc_timestamp, to_utc_timestamp, etc.

